With this notebook, you can obtain both positive and negative summarizations with Amazon product review input. To obtain better result, it is recommended to input at least 10 product reviews.

In [1]:
# This notebook is based on Google Colab. Please follow the important instructions
# in the notebook to avoid bugs.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Download SUM-QE
# Note: If you do not use colab pro and run out of RAM at SUM-QE, please comment
# the following if you rerun the whole notebook
!git clone https://github.com/nlpaueb/SumQE.git
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q1_Multi%20Task-1.h5
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q2_Multi%20Task-1.h5
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q3_Multi%20Task-1.h5
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q4_Multi%20Task-1.h5
!wget https://archive.org/download/sum-qe/BERT_DUC_all_Q5_Multi%20Task-1.h5

In [4]:
!pip uninstall -y tensorflow
!pip uninstall -y h5py
!pip install tensorflow==1.15
!pip install h5py==2.10.0
!pip install -r SumQE/requirements.txt
!pip install keras==2.3.1

Found existing installation: tensorflow 1.15.0
Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0
Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Successfully uninstalled h5py-2.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
  Using cached h5py-3.7.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Attempting uninstall: h5py
    Found existing installation:

In [5]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, SequentialSampler
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import PegasusForConditionalGeneration, AutoTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Please enter the reviews here as string 
# Recommendation: To achieve the best result, please enter at least 10 reviews.
# Recommendation: Please pick both high rating and low rating reviews

reviews = []
reviews.append("Professional 2200W high power, it is very strong, and it blows quickly. it has 3 temperature settings, 2 fan speeds and a cool shot button. This is my second Trezoro and I really like it because it is the only hair dryer that seems to prevent a lot of static.")
reviews.append("After two months my product broke. The plug for the hairdryer had an electric issue and short wired cause sparks to fly and for my hair dryer to break. I called Amazon and they assured me I had a one year warranty and gave me the email to contact Trezoro. I emailed them 3 different times in the span of a week and they have ignored me. Not okay! Treat your customers appropriately! For a 50 dollar hair dryer it should last longer and you should follow through on your warranty!")
reviews.append("I love that this blow dried my hair in minutes. Doesn’t make my hair frizzy and Greta hold when styling it. The only thing I wish it was a little shorter in length and lighter. It’s very tiring. Overall it’s a great dryer!!")
reviews.append("This is the worst hairdryer that I've ever used. It took forever to dry my hair and it caused so much frizz! I only bought it to use with my black orchid diffuser and I regret wasting my money on it. Its only positive feature is that it fits this diffuser. I'm glad to say that I did find a great dryer, the conair Infinitipro 1875 (the cheap rust orange one) and this one does dry my hair rapidly and I have far less frizz with it. While it didn't fit my black orchid, it can be adjusted with a snozzlepro or it can fit my adjustable segbeauty diffuser. I wish I had bought that dryer from the beginning instead. It cost me less than half of what I originally spent on the trezoro (I got it for $12.99 at Ross so it was even cheaper than it goes on for here) and it performs like a high end salon dryer. If anyone is looking for a dryer to use with a diffuser like the black orchid or segbeauty get that or a conair dryer instead. Again it can be adjusted with a snozzlepro or removing the inner ring from inside the diffuser.")
reviews.append("I had this product one year, and was satisfied with its ability to occasionally dry my hair. After just over a year though I went to plug in the dryer and screws fell out in my hand and the plastic crumbled. It left the reset switch inside the plug, and wires exposed. I had no choice but to throw it away. Not worth the risk of keeping around the house with kids at home. Glad it fell apart before I plugged it in, and not after. Terrible quality. Buyer beware. I wouldn’t trust this brand again. The hunt for a new hair dryer continues!")
reviews.append("I purchased this hairdryer after borrowing it from a friend and LOVING it! I purchased in December of 2020. This morning (06/09/2022) as I was using it, I noticed that the cord was getting extremely hot. In fact burning my skin when it came into contact with it. I thought that was ver strange because it take about 7 minutes to blow dry my hair and I was only 1/2 way finished. Seconds later black smoke started rolling out where the cord attaches to the device. I could hear the wires sizzling and the smell of an electrical fire filled the bathroom. Of course, I immediately unplugged it and quit using it. I followed up with my friend who recommended it and she said she had issues with hers to around the two year mark. I would be cautious purchasing this. I do plan to contact the company (if I can find their contact information).")
reviews.append("Purchased this blow dryer after researching. This one seemed to check all of the boxes. When it arrived, it did NOT come with any styling attachment as mentioned in the description. I tried to use if for the first time and the plug got EXTREMELY hot. After looking at reviews again, it seems I am not the only one to experience this issue. Happy I realized quickly before something worse happened.")
reviews.append("First, I'd like to say this is one of the best hair dryers I've ever had. Second it definitely very quiet! It drys my hair super fast BUT it's really NOT for me for the following reasons. I have very fine and thin hair..with that in mind the dryer has only 2 speeds 'High' and 'Medium High' so, it blows my hair everywhere and because it's fine it tangles easier. Also, it makes it extremely challenging to style my hair while drying. It has 3 heat settings so why not 3 speeds? If you have alot of hair, or thick hair this dryer is great. It us NOT 'Travel' size...it's the same size as every other dryer on the market but it feels great with the silicone covering and its light weight. I'm keeping it to use on my dogs because they have thick hair and I bathe them often and it works great for them. If they made one with a lower speed you'd have to go along way to find this quality for the price.")
reviews.append("This is the second one of these hair dryers that I have purchased. I love the dryer because it dries my thick curly hair quickly and smoothly. It gets hot with a lot of air volume, which is what I need to smooth out my hair quickly. The only downside is that the dryer lasted just two years before it burned out. And maybe that is the price to pay for a dryer that does get pretty hot. But I liked it well enough to buy another one!")
reviews.append("The main thing is that the hair dryer is easy to use, feels good in my hand, and gets the job done as advertised. Oh one more little suggestion— I use the nozzle attachment, and when I switch sides, I need to rotate it a little bit so the airflow is pointing at the correct angle. Due to the awesome matte finish, it’s kinda hard to rotate the attachment so I end up having to get a better hold of it to rotate, which tends to burn my hand a little. Perhaps a band of smoother material in just that area would make it easier to change direction. On the other hand, it’s nice that it retains its position and doesn’t easily get knocked out of place or fall of when I put it away in the drawer. Also, not important enough to change my review or overall opinion, just a little feedback for your design team!")

In [7]:
df_dict = {'reviews': reviews}
df_raw_reviews = pd.DataFrame(df_dict)

In [8]:
# Settings
# Please enter the path of models here
bert_path = '/content/drive/MyDrive/GR/saved/finetuned_BERT_large_uncased_0.983_0.966.model'

#If you predownload SUM-QE models, please also enter the path of the models here
sum_qe_path = ''

# After preprocessing the reviews, we save the classified sentences (positive and negative) here
# Summarization is also saved here
save_path = '/content/drive/MyDrive/GR/classified review'

# Set confidence limit here, the default value is 0.8
# The higher this limit is, the more accurate the BERT classifier is
confidence_limit = 0.8

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', 
                                          do_lower_case=True)

label_dict = {'neu': 0, 'pos': 1, 'neg': 2}
label_dict_inverse = {v: k for k, v in label_dict.items()}

# GPU is needed here. The free GPU of Google Colab would be enough.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained("bert-large-uncased",
                                                       num_labels = len(label_dict),
                                                       output_attentions = False,
                                                       output_hidden_states = False)

model.to(device)
model.load_state_dict(torch.load(bert_path, map_location=torch.device('cpu')))

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

<All keys matched successfully>

In [10]:
def evaluate(dataloader_raw):

    model.eval()
    predictions, true_vals = [], []
    
    for batch in dataloader_raw:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    predictions = np.concatenate(predictions, axis=0)
            
    return predictions

In [11]:
def softmax(x, axis=1):
    row_max = x.max(axis=axis)
    row_max = row_max.reshape(-1, 1)
    hatx = x - row_max
    hatx_exp = np.exp(hatx)
    hatx_sum = np.sum(hatx_exp, axis=axis, keepdims=True)
    s = hatx_exp / hatx_sum
    return s

In [12]:
# Do sentence tokenize and lower case
df_raw_reviews['reviews'] = df_raw_reviews['reviews'].map(lambda x:sent_tokenize(x.lower()))
df_raw_reviews = df_raw_reviews.explode('reviews')

# Make data
encoded_data_raw = tokenizer.batch_encode_plus(df_raw_reviews.reviews.values, 
                                               add_special_tokens=True, 
                                               return_attention_mask=True, 
                                               pad_to_max_length=True, 
                                               max_length=256, 
                                               return_tensors='pt'
                                               )

input_ids_raw = encoded_data_raw['input_ids']
attention_masks_raw = encoded_data_raw['attention_mask']
dataset_raw = TensorDataset(input_ids_raw, attention_masks_raw)

batch_size = 8
dataloader_raw = DataLoader(dataset_raw, 
                            sampler=SequentialSampler(dataset_raw), 
                            batch_size=batch_size)
    
# Get the labels
predictions = evaluate(dataloader_raw)
preds_flat = np.argmax(predictions, axis=1).flatten()     # pos: 1, neg: 2, neu: 0(we do not care about neu)
confidence = np.max(softmax(predictions), axis=1).flatten()

sentiment_raw = []
confidence_raw = []
for j in range(len(preds_flat)):
    sentiment_raw.append(label_dict_inverse[preds_flat[j]])
    confidence_raw.append(confidence[j])

# Add a column of sentiment classified by BERT model trained in the last part
df_raw_reviews.loc[:, 'sentiment'] = sentiment_raw
df_raw_reviews.loc[:, 'confidence'] = confidence_raw
df_raw_reviews = df_raw_reviews[df_raw_reviews['sentiment'].values != 'neu']
df_raw_reviews = df_raw_reviews[df_raw_reviews['confidence'].values >= confidence_limit]

# Reset index
df_raw_reviews.reset_index(inplace=True)
del df_raw_reviews['index']

df_raw_reviews_pos = df_raw_reviews[df_raw_reviews['sentiment'] == 'pos']
df_raw_reviews_neg = df_raw_reviews[df_raw_reviews['sentiment'] == 'neg']
df_raw_reviews_pos = df_raw_reviews_pos[df_raw_reviews_pos['confidence'] >= confidence_limit]
df_raw_reviews_neg = df_raw_reviews_neg[df_raw_reviews_neg['confidence'] >= confidence_limit]

# Concatenate sentences to str
list_pos = [item for item in df_raw_reviews_pos['reviews']]
str_pos = ' '.join(list_pos)
list_neg = [item for item in df_raw_reviews_neg['reviews']]
str_neg = ' '.join(list_neg)

# Save the sentences to txt
with open(f'{save_path}/pos.txt', 'w') as f:
    f.write(str_pos)
    f.close()

with open(f'{save_path}/neg.txt', 'w') as f:
    f.write(str_neg)
    f.close()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [13]:
# Define PEGASUS
model_name = 'google/pegasus-large'
device = "cuda" if torch.cuda.is_available() else "cpu"
sum_tokenizer = AutoTokenizer.from_pretrained(model_name)
sum_model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

# Load sentences
with open(f'{save_path}/pos.txt') as f:
    for eachline in f:
        txt_pos = eachline

with open(f'{save_path}/neg.txt') as f:
    for eachline in f:
        txt_neg = eachline

In [14]:
with open(f'{save_path}/sum_pos.txt', 'w') as f:
    batch = sum_tokenizer(txt_pos, truncation=True, padding="longest", return_tensors="pt").to(device)
    translated = sum_model.generate(**batch)
    sum_text = sum_tokenizer.batch_decode(translated, skip_special_tokens=True)
    f.write(sum_text[0])

with open(f'{save_path}/sum_neg.txt', 'w') as f:
    batch = sum_tokenizer(txt_neg, truncation=True, padding="longest", return_tensors="pt").to(device)
    translated = sum_model.generate(**batch)
    sum_text = sum_tokenizer.batch_decode(translated, skip_special_tokens=True)
    f.write(sum_text[0])

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


It is possbile that colab runs out of RAM if you run this notebook from beginning. If that happens, please restart kernel and run from here. (We have saved summarizations in the last step.)

In [1]:
# This should be the same as the beginning

# If you predownload SUM-QE models, please also enter the path of the models here
sum_qe_path = ''

# After preprocessing the reviews, we save the classified sentences (positive and negative) here
# Summarization is also saved here
save_path = '/content/drive/MyDrive/GR/classified review'

In [2]:
import sys
sys.path.insert(0,'/content/SumQE')
%cd /content/SumQE/

import nltk
nltk.download('punkt')

# src/examples.py
import numpy as np
import pandas as pd

from keras.models import load_model
from nltk.tokenize import sent_tokenize

from src.BERT_experiments.BERT_model import BERT, custom_loss, set_quality_index
from src.vectorizer import BERTVectorizer

/content/SumQE


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


In [3]:
# Turn scores to "good", "fair", and "bad"
# The value comes from the corresponding paper
def measure_quality(name, score, sentiment):
    if name == 1 and sentiment == 'pos':
        if score < 0.2504:
            quality = 'bad'
        if score > 0.3704:
            quality = 'good'
        else:
            quality = 'fair'

    if name == 2 and sentiment == 'pos':
        if score < 0.2543:
            quality = 'bad'
        if score > 0.3737:
            quality = 'good'
        else:
            quality = 'fair'

    if name == 3 and sentiment == 'pos':
        if score < 0.2515:
            quality = 'bad'
        if score > 0.371:
            quality = 'good'
        else:
            quality = 'fair'

    if name == 4 and sentiment == 'pos':
        if score < 0.1044:
            quality = 'bad'
        if score > 0.2692:
            quality = 'good'
        else:
            quality = 'fair'

    if name == 5 and sentiment == 'pos':
        if score < 0.1111:
            quality = 'bad'
        if score > 0.2737:
            quality = 'good'
        else:
            quality = 'fair'

    if name == 1 and sentiment == 'neg':
        if score < 0.3597:
            quality = 'bad'
        if score > 0.5103:
            quality = 'good'
        else:
            quality = 'fair'

    if name == 2 and sentiment == 'neg':
        if score < 0.3611:
            quality = 'bad'
        if score > 0.5111:
            quality = 'good'
        else:
            quality = 'fair'

    if name == 3 and sentiment == 'neg':
        if score < 0.358:
            quality = 'bad'
        if score > 0.5089:
            quality = 'good'
        else:
            quality = 'fair'

    if name == 4 and sentiment == 'neg':
        if score < 0.2423:
            quality = 'bad'
        if score > 0.4681:
            quality = 'good'
        else:
            quality = 'fair'

    if name == 5 and sentiment == 'neg':
        if score < 0.2449:
            quality = 'bad'
        if score > 0.4707:
            quality = 'good'
        else:
            quality = 'fair'
    return quality

In [4]:
# Quality evaluation
scores = []
Qs = []
senti = []
quality = []
sentiment_range = ['pos', 'neg']
for i in range(1, 6):
    print('[INFO] Running for Q{}'.format(i))
    QUALITY = 'Q{}'.format(i)
    model_path = '/content/BERT_DUC_all_{}_Multi Task-1.h5'.format(QUALITY)

    # Set the quality index used in custom_loss
    set_quality_index(mode = 'Multi Task-1', quality = QUALITY)

    # Load the model
    model = load_model(model_path, custom_objects={'BERT': BERT, 'custom_loss': custom_loss})

    # Define the vectorizer
    vectorizer = BERTVectorizer()

    for j in range(2):
        sentiment = sentiment_range[j]
        # For each model, do scoring
        Qs.append('Q{}'.format(i))
        senti.append(sentiment)
        with open(f'{save_path}/sum_{sentiment}.txt') as f:
            for eachline in f:
                after_sum = eachline

        # Summarization
        summary_token_ids = []
        for k, sentence in enumerate(sent_tokenize(after_sum)):
            sentence_tok = vectorizer.vectorize_inputs(sequence = sentence, i = k)
            summary_token_ids = summary_token_ids + sentence_tok
                
        # Transform the summary_tokens_ids into inputs --> (bpe_ids, mask, segments)
        inputs = vectorizer.transform_to_inputs(summary_token_ids)

        # Construct the dict that you will feed on your network. If you have multiple summaries,
        # you can update the lists and feed all of them together.
        test_dict = {
            'word_inputs': np.asarray([inputs[0, 0]]),
            'pos_inputs': np.asarray([inputs[1, 0]]),
            'seg_inputs': np.asarray([inputs[2, 0]])
        }
              
        output = model.predict(test_dict, batch_size=1)
        quality_score = round(output[0][0], 4)
        scores.append(quality_score)
        quality.append(measure_quality(i, quality_score, sentiment))

[INFO] Running for Q1


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "






[INFO] Running for Q2


/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[INFO] Running for Q3


/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[INFO] Running for Q4


/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[INFO] Running for Q5


/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [5]:
scores_df = pd.DataFrame({'QE score':scores,'measurement':Qs, 'sentiment':senti, 'quality': quality})
scores_df.to_csv('/content/drive/MyDrive/GR/scores_test.csv', sep = ',', index = True)
scores_df = scores_df.sort_values(by=["sentiment", "measurement"], ascending=[False, True])

Let's see the results!

In [6]:
with open(f'{save_path}/sum_pos.txt') as f:
            for eachline in f:
                sum_pos = eachline

with open(f'{save_path}/sum_neg.txt') as f:
            for eachline in f:
                sum_neg = eachline

In [7]:
sum_pos

"i'm glad to say that i did find a great dryer, the conair infinitipro 1875 (the cheap rust orange one) and this one does dry my hair rapidly and i have far less frizz with it. it cost me less than half of what i originally spent on the trezoro (i got it for $12.99 at ross so it was even cheaper than it goes on for here) and it performs like a high end salon dryer. i have very fine and thin hair..with that in mind the dryer has only 2 speeds 'high' and'medium high' so, it blows my hair everywhere and because it's fine it tangles easier."

In [8]:
sum_neg

'the plug for the hairdryer had an electric issue and short wired cause sparks to fly and for my hair dryer to break.'

In [9]:
scores_df

,QE score,measurement,sentiment,quality
0,0.4939,Q1,pos,good
2,0.4968,Q2,pos,good
4,0.4955,Q3,pos,good
6,0.4362,Q4,pos,good
8,0.4424,Q5,pos,good
1,0.4741,Q1,neg,fair
3,0.4759,Q2,neg,fair
5,0.4739,Q3,neg,fair
7,0.4910,Q4,neg,good
9,0.4935,Q5,neg,good
